# Reading general data of the problems


In [1]:
# coding=utf-8
from __future__ import division
import json
import os
import sys
import multiprocessing

from MyUtils import clean_folder, read_files, shuffle_docs
from Word2Dim import Word2Dim

dataset_path = '.' + os.sep + 'pan19-cross-domain-authorship-attribution-training-dataset-2019-01-23'
outpath = '.' + os.sep + 'dev_out'

clean_folder(outpath)

infocollection = dataset_path + os.sep + 'collection-info.json'
problems = []
language = []
with open(infocollection, 'r') as f:
    for attrib in json.load(f):
        problems.append(attrib['problem-name'])
        language.append(attrib['language'])
print('Reading general data of the problems, done!')


Reading general data of the problems, done!


# Reading problem 1

In [ ]:

problem = problems[0]
index = 0

# used for n_gram extraction and word indexing, a threshold which prevent words appearing lower than this value to be counted in calculations
tf = 5


infoproblem = dataset_path + os.sep + problem + os.sep + 'problem-info.json'
candidates = []
with open(infoproblem, 'r') as f:
    fj = json.load(f)
    unk_folder = fj['unknown-folder']
    for attrib in fj['candidate-authors']:
        candidates.append(attrib['author-name'])

candidates.sort()
# Building training set
train_docs = []
for candidate in candidates:
    train_docs.extend(read_files(dataset_path + os.sep + problem, candidate))
train_texts = [text for i, (text, label) in enumerate(train_docs)]
train_labels = [label for i, (text, label) in enumerate(train_docs)]
train_texts, train_labels = shuffle_docs(train_texts, train_labels)
index_2_label_dict = {i: l for i, l in enumerate(set(train_labels))}
label_2_index_dict = {l: i for i, l in enumerate(set(train_labels))}
train_labels = [label_2_index_dict[v] for v in train_labels]
w2d = Word2Dim(lang= language[index])
train_tokenized_with_pos, train_tokenized_indexed = w2d.fit_transform_texts(train_texts, train_labels, tf= tf)

maxlen = len(max(train_tokenized_indexed, key=len))  # We will cut the texts after # words
embedding_dim = w2d.word_embedding.shape[1]

# preparing test set
ground_truth_file = dataset_path + os.sep + problem + os.sep + 'ground-truth.json'
gt = {}
with open(ground_truth_file, 'r') as f:
    for attrib in json.load(f)['ground_truth']:
        gt[attrib['unknown-text']] = attrib['true-author']

test_docs = read_files(dataset_path + os.sep + problem, unk_folder, gt)
test_texts = [text for i, (text, label) in enumerate(test_docs)]
test_labels = [label for i, (text, label) in enumerate(test_docs)]

# Filter validation to known authors
test_texts = [text for i, (text, label) in enumerate(test_docs) if label in label_2_index_dict.keys()]
test_labels = [label for i, (text, label) in enumerate(test_docs) if label in label_2_index_dict.keys()]

test_labels = [label_2_index_dict[v] for v in test_labels]

test_tokenized_with_pos, test_tokenized_indexed = w2d.transform(test_texts)
print("Reading problem 1, done!")

doc count to process:  819
Processing doc # 1
Processing doc # 2
Processing doc # 3
Processing doc # 4
Processing doc # 5
Processing doc # 6
Processing doc # 7
Processing doc # 8
Processing doc # 9
Processing doc # 10
Processing doc # 11
Processing doc # 12
Processing doc # 13
Processing doc # 14
Processing doc # 15
Processing doc # 16
Processing doc # 17
Processing doc # 18
Processing doc # 19
Processing doc # 20
Processing doc # 21
Processing doc # 22
Processing doc # 23
Processing doc # 24
Processing doc # 25
Processing doc # 26
Processing doc # 27
Processing doc # 28
Processing doc # 29
Processing doc # 30
Processing doc # 31
Processing doc # 32
Processing doc # 33
Processing doc # 34
Processing doc # 35
Processing doc # 36
Processing doc # 37
Processing doc # 38
Processing doc # 39
Processing doc # 40
Processing doc # 41


# Data Extraction for Neural Net



In [ ]:
from MyUtils import extract_n_grams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

n = 3
vocabulary = extract_n_grams(train_docs, n, tf)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n), lowercase=False, vocabulary=vocabulary)
n_gram_train_data = vectorizer.fit_transform(train_texts)

n_gram_train_data = n_gram_train_data.astype(float)

for i, v in enumerate(train_texts):
    n_gram_train_data[i] = n_gram_train_data[i] / len(train_texts[i])
n_gram_test_data = vectorizer.transform(test_texts)
n_gram_test_data = n_gram_test_data.astype(float)
for i, v in enumerate(test_texts):
    n_gram_test_data[i] = n_gram_test_data[i] / len(test_texts[i])
max_abs_scaler = preprocessing.MaxAbsScaler()
scaled_train_data_ngrams = max_abs_scaler.fit_transform(n_gram_train_data)
scaled_test_data_ngrams = max_abs_scaler.transform(n_gram_test_data)
max_abs_scaler = preprocessing.MaxAbsScaler()
scaled_train_data_words = max_abs_scaler.fit_transform(w2d.get_texts_vectorized_and_normalized(train_tokenized_indexed)[:, 1:])
scaled_test_data_words = max_abs_scaler.transform(w2d.get_texts_vectorized_and_normalized(test_tokenized_indexed)[:, 1:])
print("Done!")

In [ ]:
print(scaled_train_data_words.shape)
print(scaled_test_data_words.shape)
print(len(w2d.word_index))

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


from keras import layers, Input, callbacks
from keras.models import Sequential, Model
from keras import optimizers, regularizers
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

callbacks_list_neu = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=50,
    ),
    callbacks.ModelCheckpoint(
        filepath='my_model_neu.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        verbose=1, 
        patience=20,
    )
]

callbacks_list_neu_ngrams = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=50,
    ),
    callbacks.ModelCheckpoint(
        filepath='my_model_neu_ngrams.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        verbose=1, 
        patience=20,
    )
]

callbacks_list_neu_words = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=50,
    ),
    callbacks.ModelCheckpoint(
        filepath='my_model_neu_words.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        verbose=1, 
        patience=20,
    )
]

callbacks_list_convnet = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=35,
    ),
    callbacks.ModelCheckpoint(
        filepath='my_model_convnet.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        verbose=1, 
        patience=10,
    )
]

callbacks_list_stacked = [
    callbacks.ModelCheckpoint(
        filepath='my_model_neu.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        verbose=1, 
        patience=20,
    )
]

train_data = pad_sequences(train_tokenized_indexed, maxlen=maxlen)

test_data = pad_sequences(test_tokenized_indexed, maxlen=maxlen)

X_train, X_val, y_train, y_val = train_data, test_data, to_categorical(train_labels), to_categorical(test_labels)
# X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels,
#                                                   test_size=len(set(train_labels)), random_state=2019,
#                                                   stratify=train_labels)

# y_train = to_categorical(y_train)
# y_val = to_categorical(y_val)
# print(X_train.shape)

print(scaled_train_data_ngrams.shape)


In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


from keras import layers, Input
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


neu_ng = Sequential()
neu_ng.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                               input_shape=(scaled_train_data_ngrams.shape[1],)))
neu_ng.add(layers.Dropout(0.3))
neu_ng.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu_ng.add(layers.Dropout(0.3))
neu_ng.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu_ng.add(layers.Dropout(0.3))
neu_ng.add(layers.Dense(len(set(train_labels)), activation='softmax'))

neu_ng.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
neu_ng.summary()
history = neu_ng.fit(scaled_train_data_ngrams, y_train,
                    validation_data=(scaled_test_data_ngrams, y_val),
                    epochs=2000,
                    batch_size=1,
                    callbacks=callbacks_list_neu_ngrams,
                    verbose= 1
                   )
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
print(max(val_acc))


In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


from keras import layers, Input
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


neu_wo = Sequential()
neu_wo.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                               input_shape=(scaled_train_data_words.shape[1],)))
neu_wo.add(layers.Dropout(0.3))
neu_wo.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu_wo.add(layers.Dropout(0.3))
neu_wo.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu_wo.add(layers.Dropout(0.3))
neu_wo.add(layers.Dense(len(set(train_labels)), activation='softmax'))


neu_wo.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
neu_wo.summary()
history = neu_wo.fit(scaled_train_data_words, y_train,
                    validation_data=(scaled_train_data_words, y_val),
                    epochs=2000,
                    batch_size=1,
                    callbacks=callbacks_list_neu_words,
                    verbose= 1
                   )
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
print(max(val_acc))

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


from keras import layers, Input
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

word_input_tensor = Input(shape=(scaled_train_data_words.shape[1],) , name='words')
ngram_input_tensor = Input(shape=(scaled_train_data_ngrams.shape[1],) , name='n_grams')

neu0 = Sequential()
neu0.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                               input_shape=(scaled_train_data_ngrams.shape[1],)))
neu0.add(layers.Dropout(0.3))
neu0.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu0.add(layers.Dropout(0.3))
neu0.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu0.add(layers.Dropout(0.3))

neu1 = Sequential()
neu1.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                               input_shape=(scaled_train_data_words.shape[1],)))
neu1.add(layers.Dropout(0.3))
neu1.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu1.add(layers.Dropout(0.3))
neu1.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
neu1.add(layers.Dropout(0.3))

output_tensor_0 = neu0(ngram_input_tensor)
output_tensor_1 = neu1(word_input_tensor)


conv_input_tensor = Input(shape=(maxlen,) , name='convnets')

conv_1d_s3_model = Sequential()
conv_1d_s3_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_s3_model.add(layers.Dropout(0.2))
conv_1d_s3_model.add(layers.SeparableConv1D(32, 3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_s3_model.add(layers.BatchNormalization())
conv_1d_s3_model.add(layers.GlobalMaxPooling1D())

conv_output_tensor_0 = conv_1d_s3_model(conv_input_tensor)


conv_1d_s1_model = Sequential()
conv_1d_s1_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_s1_model.add(layers.Dropout(0.2))
conv_1d_s1_model.add(layers.SeparableConv1D(32, 1, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_s1_model.add(layers.BatchNormalization())
conv_1d_s1_model.add(layers.GlobalMaxPooling1D())

conv_output_tensor_1 = conv_1d_s1_model(conv_input_tensor)

conv_1d_complex_model = Sequential()
conv_1d_complex_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_complex_model.add(layers.Dropout(0.2))
conv_1d_complex_model.add(layers.SeparableConv1D(32, 2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_complex_model.add(layers.BatchNormalization())
conv_1d_complex_model.add(layers.GlobalMaxPooling1D())

conv_output_tensor_2 = conv_1d_complex_model(conv_input_tensor)



concatenated = layers.concatenate([output_tensor_0,
                                   output_tensor_1,
                                   conv_output_tensor_0,
                                   conv_output_tensor_1,
                                   conv_output_tensor_2,
                                  ], axis=-1)

concatenated = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(concatenated)
concatenated = layers.Dropout(0.3)(concatenated)
concatenated = layers.Dense(len(set(train_labels)), activation='softmax')(concatenated)

model = Model([ngram_input_tensor, word_input_tensor, conv_input_tensor], concatenated)
model.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()
history = model.fit([scaled_train_data_ngrams, scaled_train_data_words, X_train], y_train,
                    validation_data=([scaled_test_data_ngrams, scaled_test_data_words, X_val], y_val),
                    epochs=2000,
                    batch_size=1,
                    callbacks=callbacks_list_neu,
                    verbose= 1
                   )
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
print(max(val_acc))

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


from keras import layers, Input
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split



train_data = pad_sequences(train_tokenized_indexed, maxlen=maxlen)

test_data = pad_sequences(test_tokenized_indexed, maxlen=maxlen)

X_train, X_val, y_train, y_val = train_data, test_data, to_categorical(train_labels), to_categorical(test_labels)
# X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels,
#                                                   test_size=0.28, random_state=2019,
#                                                   stratify=train_labels)

# y_train = to_categorical(y_train)
# y_val = to_categorical(y_val)
convnet_input_tensor = Input(shape=(maxlen,) , name='convnet_words')

conv_1d_s3_model = Sequential()
conv_1d_s3_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_s3_model.add(layers.Dropout(0.3))
conv_1d_s3_model.add(layers.SeparableConv1D(64, 3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_s3_model.add(layers.BatchNormalization())
conv_1d_s3_model.add(layers.GlobalMaxPooling1D())
# conv_1d_s3_model.add(layers.BatchNormalization())
# conv_1d_s3_model.add(layers.Dense(len(set(train_labels)), activation='softmax'))

# conv_1d_s3_model.layers[0].set_weights([w2d.word_embedding])
# conv_1d_s3_model.layers[0].trainable = False
conv_output_tensor_0 = conv_1d_s3_model(convnet_input_tensor)


conv_1d_s1_model = Sequential()
conv_1d_s1_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_s1_model.add(layers.Dropout(0.3))
conv_1d_s1_model.add(layers.SeparableConv1D(64, 1, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_s1_model.add(layers.BatchNormalization())
# conv_1d_s1_model.add(layers.MaxPooling1D(2))
# conv_1d_s1_model.add(layers.SeparableConv1D(32, 3, activation='relu'))
conv_1d_s1_model.add(layers.GlobalMaxPooling1D())
# conv_1d_s1_model.add(layers.BatchNormalization())
# conv_1d_s1_model.add(layers.Dense(len(set(train_labels)), activation='softmax'))

# conv_1d_s1_model.layers[0].set_weights([w2d.word_embedding])
# conv_1d_s1_model.layers[0].trainable = False
conv_output_tensor_1 = conv_1d_s1_model(convnet_input_tensor)

conv_1d_complex_model = Sequential()
conv_1d_complex_model.add(layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen))
conv_1d_complex_model.add(layers.Dropout(0.3))
conv_1d_complex_model.add(layers.SeparableConv1D(64, 2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
conv_1d_complex_model.add(layers.BatchNormalization())
# conv_1d_complex_model.add(layers.SeparableConv1D(64, 3, activation='relu'))
# conv_1d_complex_model.add(layers.MaxPooling1D(2))
# conv_1d_complex_model.add(layers.SeparableConv1D(64, 3, activation='relu'))
# conv_1d_complex_model.add(layers.SeparableConv1D(128, 3, activation='relu'))
# conv_1d_complex_model.add(layers.MaxPooling1D(2))
# conv_1d_complex_model.add(layers.SeparableConv1D(64, 3, activation='relu'))
# conv_1d_complex_model.add(layers.SeparableConv1D(128, 3, activation='relu'))
conv_1d_complex_model.add(layers.GlobalMaxPooling1D())
# conv_1d_complex_model.add(layers.BatchNormalization())
# conv_1d_s1_model.add(layers.Dense(len(set(train_labels)), activation='softmax'))

# conv_1d_complex_model.layers[0].set_weights([w2d.word_embedding])
# conv_1d_complex_model.layers[0].trainable = False
conv_output_tensor_2 = conv_1d_complex_model(convnet_input_tensor)

# x = layers.Embedding(w2d.word_embedding.shape[0], embedding_dim, input_length=maxlen)(word_input_tensor)
# x = layers.Conv1D(128, 5, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)
# x = layers.Dropout(0.2)(x)
# x = layers.BatchNormalization()(x)
# y = layers.Conv1D(128, 10, activation='relu', padding='same')(x)
# added = layers.add([y, x])
# added = layers.GlobalMaxPooling1D()(added)

concatenated = layers.concatenate([conv_output_tensor_0,
                                   conv_output_tensor_1,
                                   conv_output_tensor_2,
#                                    ,added
                                  ], axis=-1)
concatenated = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(concatenated)
concatenated = layers.Dropout(0.3)(concatenated)
concatenated = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(concatenated)
concatenated = layers.Dropout(0.3)(concatenated)
answer = layers.Dense(len(set(train_labels)), activation='softmax')(concatenated)

model = Model(convnet_input_tensor, answer)
model.summary()

# model.layers[0].set_weights([w2d.word_embedding])
# model.layers[0].trainable = False

model.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=2000,
                    batch_size=1,
                    callbacks=callbacks_list_convnet,
                    verbose= 2
                   )

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
print(max(val_acc))

In [ ]:
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from keras.utils import plot_model
from keras.layers.merge import concatenate

# load models from file
def load_all_models(model_names_list):
    all_models = list()
    for model_name in model_names_list:
        # define filename for this ensemble
#         filename = 'models/model_' + str(i + 1) + '.h5'
        filename = model_name + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models


# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer.name = 'ensemble_' + str(i + 1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
#     print(ensemble_visible)
#     ensemble_visible = [[ngram_input_tensor, word_input_tensor], convnet_input_tensor]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
#     ensemble_outputs = [concatenated, answer]
    merge = concatenate(ensemble_outputs)
    hidden = layers.Dense(128, activation='relu')(merge)
    hidden = layers.Dropout(0.3)(hidden)
    output = layers.Dense(len(set(train_labels)), activation='softmax')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
#     plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=3e-4), metrics=['accuracy'])
    return model

# fit a stacked model
def fit_stacked_model(model, inputX, inputy, valX, valy):
    # prepare input data
#     X = [inputX for _ in range(len(model.input))]
    # encode output data
#     inputy_enc = to_categorical(inputy)
    # fit model
    model.fit(inputX, inputy, validation_data=(valX, valy), callbacks=callbacks_list_stacked, epochs=300, verbose=1)
    
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
#     X = [inputX for _ in range(len(model.input))]
    # make prediction
    return model.predict(inputX, verbose=0)

print('done!')

In [ ]:
members = load_all_models(['my_model_neu_ngrams', 'my_model_neu_words', 'my_model_convnet'])
print('Loaded %d models' % len(members))
# define ensemble model
stacked_model = define_stacked_model(members)
# fit stacked model on test dataset
fit_stacked_model(stacked_model, [scaled_train_data_ngrams, scaled_train_data_words, X_train], y_train, [scaled_test_data_ngrams, scaled_test_data_words, X_val], y_val)


In [ ]:
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from keras.utils import plot_model
from keras.layers.merge import concatenate

model_neu = load_model('my_model_neu.h5')
model_conv = load_model('my_model_convnet.h5')

for layer in model_neu.layers:
    # make not trainable
    layer.trainable = False
    
for layer in model_conv.layers:
    # make not trainable
    layer.trainable = False

ensemble_visible = [ngram_input_tensor, word_input_tensor, convnet_input_tensor]